In [1]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [2]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [3]:
%%sql
CREATE TABLE Member (
    MemberID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Address TEXT,
    Email TEXT UNIQUE,
    Phone TEXT UNIQUE,
    MembershipType TEXT
);

CREATE TABLE Item (
    ItemID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    Genre TEXT,
    Type TEXT NOT NULL,
    Availability BOOLEAN NOT NULL
);

CREATE TABLE Book (
    ISBN TEXT PRIMARY KEY,
    Title TEXT NOT NULL,
    Author TEXT NOT NULL,
    Publisher TEXT,
    Year INTEGER,
    Edition TEXT,
    ItemID INTEGER UNIQUE,
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID)
);

CREATE TABLE Multimedia (
    MediaID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    Artist TEXT,
    Type TEXT,
    Genre TEXT,
    ItemID INTEGER UNIQUE,
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID)
);

CREATE TABLE Borrow (
    BorrowID INTEGER PRIMARY KEY,
    MemberID INTEGER,
    ItemID INTEGER,
    BorrowDate DATE NOT NULL,
    DueDate DATE NOT NULL,
    ReturnDate DATE,
    FOREIGN KEY (MemberID) REFERENCES Member(MemberID),
    FOREIGN KEY (ItemID) REFERENCES Item(ItemID)
);

CREATE TABLE Fine (
    FineID INTEGER PRIMARY KEY,
    MemberID INTEGER,
    Amount DECIMAL(10,2) NOT NULL,
    DueDate DATE NOT NULL,
    PaidStatus BOOLEAN NOT NULL,
    FOREIGN KEY (MemberID) REFERENCES Member(MemberID)
);

CREATE TABLE Event (
    EventID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Type TEXT,
    Date DATE NOT NULL,
    Time TEXT,
    AudienceType TEXT
);

CREATE TABLE Room (
    RoomID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Capacity INTEGER NOT NULL,
    Location TEXT
);

CREATE TABLE Personnel (
    PersonnelID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Role TEXT NOT NULL,
    ContactDetails TEXT
);

CREATE TABLE ItemRecord (
    RecordID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    ItemType TEXT,
    DateAdded DATE NOT NULL
);

CREATE TABLE Attends (
    MemberID INTEGER,
    EventID INTEGER,
    PRIMARY KEY (MemberID, EventID),
    FOREIGN KEY (MemberID) REFERENCES Member(MemberID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID)
);

CREATE TABLE Manages (
    PersonnelID INTEGER,
    EventID INTEGER,
    PRIMARY KEY (PersonnelID, EventID),
    FOREIGN KEY (PersonnelID) REFERENCES Personnel(PersonnelID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID)
);

CREATE TABLE Held (
    EventID INTEGER,
    RoomID INTEGER,
    PRIMARY KEY (EventID, RoomID),
    FOREIGN KEY (EventID) REFERENCES Event(EventID),
    FOREIGN KEY (RoomID) REFERENCES Room(RoomID)
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [4]:
%%sql
-- Prevent borrowing an item that is not available
CREATE TRIGGER PreventUnavailableBorrow
BEFORE INSERT ON Borrow
FOR EACH ROW
WHEN (SELECT Availability FROM Item WHERE ItemID = NEW.ItemID) = 0
BEGIN
    SELECT RAISE(ABORT, 'Item is not available for borrowing');
END;

-- Automatically update item availability when borrowed
CREATE TRIGGER UpdateItemAvailabilityBorrow
AFTER INSERT ON Borrow
FOR EACH ROW
BEGIN
    UPDATE Item SET Availability = 0 WHERE ItemID = NEW.ItemID;
END;

-- Automatically update item availability when returned
CREATE TRIGGER UpdateItemAvailabilityReturn
AFTER UPDATE ON Borrow
FOR EACH ROW
WHEN NEW.ReturnDate IS NOT NULL
BEGIN
    UPDATE Item SET Availability = 1 WHERE ItemID = NEW.ItemID;
END;

-- Prevent duplicate borrowing of the same item by the same member
CREATE TRIGGER PreventDuplicateBorrow
BEFORE INSERT ON Borrow
FOR EACH ROW
WHEN EXISTS (
    SELECT 1 FROM Borrow
    WHERE MemberID = NEW.MemberID AND ItemID = NEW.ItemID AND ReturnDate IS NULL
)
BEGIN
    SELECT RAISE(ABORT, 'Member has already borrowed this item and has not returned it');
END;

-- Automatically issue a fine for overdue items
CREATE TRIGGER IssueFineForOverdue
AFTER UPDATE ON Borrow
FOR EACH ROW
WHEN NEW.ReturnDate > NEW.DueDate
BEGIN
    INSERT INTO Fine (MemberID, Amount, DueDate, PaidStatus)
    VALUES (NEW.MemberID, 5.00 * (JULIANDAY(NEW.ReturnDate) - JULIANDAY(NEW.DueDate)), NEW.ReturnDate, 0);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [5]:
%%sql
-- Insert 10 records into Member table
INSERT INTO Member (Name, Address, Email, Phone, MembershipType)
VALUES 
('John Doe', '123 Main St', 'john@example.com', '123-456-7890', 'Regular'),
('Jane Smith', '456 Oak Ave', 'jane@example.com', '987-654-3210', 'Premium'),
('Sam Wilson', '789 Pine Rd', 'sam@example.com', '555-123-4567', 'Regular'),
('Emily Davis', '101 Maple Dr', 'emily@example.com', '555-765-4321', 'Premium'),
('Michael Brown', '202 Birch Ln', 'michael@example.com', '555-234-5678', 'Regular'),
('Jessica Taylor', '303 Cedar Blvd', 'jessica@example.com', '555-876-5432', 'Premium'),
('Robert Lee', '404 Elm St', 'robert@example.com', '555-345-6789', 'Regular'),
('Olivia Clark', '505 Oak St', 'olivia@example.com', '555-567-8901', 'Premium'),
('James Walker', '606 Pine St', 'james@example.com', '555-678-9012', 'Regular'),
('Sophia Martinez', '707 Maple Rd', 'sophia@example.com', '555-789-0123', 'Premium');

-- Insert 10 records into Item table
INSERT INTO Item (Title, Genre, Type, Availability)
VALUES
('The Great Gatsby', 'Classic', 'Book', 1),
('Inception', 'Sci-Fi', 'Movie', 1),
('The Matrix', 'Action', 'Movie', 0),
('Harry Potter and the Sorcerers Stone', 'Fantasy', 'Book', 1),
('The Shawshank Redemption', 'Drama', 'Movie', 1),
('The Lord of the Rings', 'Fantasy', 'Book', 0),
('The Dark Knight', 'Action', 'Movie', 1),
('1984', 'Dystopian', 'Book', 1),
('Pulp Fiction', 'Crime', 'Movie', 0),
('The Godfather', 'Crime', 'Movie', 1);

-- Insert 10 records into Book table
INSERT INTO Book (ISBN, Title, Author, Publisher, Year, Edition, ItemID)
VALUES
('9780743273565', 'The Great Gatsby', 'F. Scott Fitzgerald', 'Scribner', 1925, '1st', 1),
('9780439554930', 'Harry Potter and the Sorcerers Stone', 'J.K. Rowling', 'Scholastic', 1997, '1st', 4),
('9780451524935', '1984', 'George Orwell', 'Signet', 1949, '1st', 8),
('9780061122415', 'The Catcher in the Rye', 'J.D. Salinger', 'Little, Brown and Company', 1951, '1st', 11),
('9781501101250', 'The Hobbit', 'J.R.R. Tolkien', 'HarperCollins', 1937, '1st', 14),
('9780451530917', 'Brave New World', 'Aldous Huxley', 'Harper Perennial Modern Classics', 1932, '1st', 16),
('9780062315007', 'The Book Thief', 'Markus Zusak', 'Alfred A. Knopf', 2005, '1st', 17),
('9780316769488', 'To Kill a Mockingbird', 'Harper Lee', 'Harper Perennial Modern Classics', 1960, '1st', 18),
('9780393972832', 'Moby-Dick', 'Herman Melville', 'W.W. Norton & Company', 1851, '1st', 19),
('9780060850524', 'The Kite Runner', 'Khaled Hosseini', 'Riverhead Books', 2003, '1st', 20);

-- Insert 10 records into Multimedia table
INSERT INTO Multimedia (Title, Artist, Type, Genre, ItemID)
VALUES
('Shape of You', 'Ed Sheeran', 'Song', 'Pop', 2),
('Blinding Lights', 'The Weeknd', 'Song', 'Pop', 3),
('Bohemian Rhapsody', 'Queen', 'Song', 'Rock', 5),
('Stairway to Heaven', 'Led Zeppelin', 'Song', 'Rock', 6),
('Rolling in the Deep', 'Adele', 'Song', 'Pop', 7),
('Hotel California', 'Eagles', 'Song', 'Rock', 8),
('Imagine', 'John Lennon', 'Song', 'Rock', 9),
('Like a Rolling Stone', 'Bob Dylan', 'Song', 'Rock', 10),
('Smells Like Teen Spirit', 'Nirvana', 'Song', 'Rock', 11),
('Sweet Child O Mine', 'Guns N Roses', 'Song', 'Rock', 12);

-- Insert 10 records into Borrow table
INSERT INTO Borrow (MemberID, ItemID, BorrowDate, DueDate, ReturnDate)
VALUES
(1, 1, '2025-03-01', '2025-03-14', '2025-03-12'),
(2, 2, '2025-03-02', '2025-03-15', NULL),
(4, 4, '2025-03-04', '2025-03-17', NULL),
(5, 5, '2025-03-05', '2025-03-18', '2025-03-17'),
(7, 7, '2025-03-07', '2025-03-20', '2025-03-19'),
(8, 8, '2025-03-08', '2025-03-21', NULL),
(10, 10, '2025-03-10', '2025-03-23', NULL);

-- Insert 10 records into Fine table
INSERT INTO Fine (MemberID, Amount, DueDate, PaidStatus)
VALUES
(1, 5.00, '2025-03-14', 0),
(2, 10.00, '2025-03-15', 1),
(3, 15.00, '2025-03-16', 0),
(4, 20.00, '2025-03-17', 1),
(5, 25.00, '2025-03-18', 0),
(6, 30.00, '2025-03-19', 1),
(7, 35.00, '2025-03-20', 0),
(8, 40.00, '2025-03-21', 1),
(9, 45.00, '2025-03-22', 0),
(10, 50.00, '2025-03-23', 1);

-- Insert 10 records into Event table
INSERT INTO Event (Name, Type, Date, Time, AudienceType)
VALUES
('Book Club', 'Meeting', '2025-03-15', '10:00 AM', 'Adults'),
('Library Workshop', 'Workshop', '2025-03-16', '11:00 AM', 'All Ages'),
('Film Screening', 'Event', '2025-03-17', '2:00 PM', 'Teens'),
('Charity Event', 'Event', '2025-03-18', '4:00 PM', 'Adults'),
('Author Talk', 'Talk', '2025-03-19', '3:00 PM', 'All Ages'),
('Poetry Reading', 'Reading', '2025-03-20', '5:00 PM', 'Adults'),
('Cooking Class', 'Class', '2025-03-21', '12:00 PM', 'All Ages'),
('Game Night', 'Event', '2025-03-22', '7:00 PM', 'Teens'),
('Concert', 'Music', '2025-03-23', '8:00 PM', 'All Ages'),
('Art Show', 'Exhibition', '2025-03-24', '10:00 AM', 'Adults');

-- Insert 10 records into Room table
INSERT INTO Room (Name, Capacity, Location)
VALUES
('Room A', 50, 'First Floor'),
('Room B', 30, 'Second Floor'),
('Room C', 20, 'Third Floor'),
('Room D', 100, 'First Floor'),
('Room E', 40, 'Second Floor'),
('Room F', 60, 'Third Floor'),
('Room G', 80, 'First Floor'),
('Room H', 25, 'Second Floor'),
('Room I', 70, 'Third Floor'),
('Room J', 90, 'First Floor');

-- Insert 10 records into Personnel table
INSERT INTO Personnel (Name, Role, ContactDetails)
VALUES
('John Smith', 'Librarian', '555-123-4567'),
('Sarah Johnson', 'Event Coordinator', '555-234-5678'),
('David Brown', 'Technician', '555-345-6789'),
('Emily White', 'Manager', '555-456-7890'),
('Chris Green', 'Assistant Librarian', '555-567-8901'),
('Daniel Black', 'Security', '555-678-9012'),
('Laura Blue', 'Assistant Manager', '555-789-0123'),
('Michael Gray', 'Technician', '555-890-1234'),
('Olivia Red', 'Coordinator', '555-901-2345'),
('James Purple', 'Security', '555-012-3456');

-- Insert 10 records into ItemRecord table
INSERT INTO ItemRecord (Title, ItemType, DateAdded)
VALUES
('The Great Gatsby', 'Book', '2025-03-01'),
('Inception', 'Movie', '2025-03-02'),
('Harry Potter and the Sorcerers Stone', 'Book', '2025-03-03'),
('The Shawshank Redemption', 'Movie', '2025-03-04'),
('The Lord of the Rings', 'Book', '2025-03-05'),
('The Dark Knight', 'Movie', '2025-03-06'),
('1984', 'Book', '2025-03-07'),
('Pulp Fiction', 'Movie', '2025-03-08'),
('The Godfather', 'Movie', '2025-03-09'),
('The Catcher in the Rye', 'Book', '2025-03-10');

Running query in 'sqlite:///library.db'

10 rows affected.

10 rows affected.

10 rows affected.

10 rows affected.

...

10 rows affected.

10 rows affected.

10 rows affected.

10 rows affected.

++
||
++
++